我认为擅长讲故事的方法应该有上面提到的四个部分组成，而且它们之间应该做到泾渭分明，而不是混杂在一起。但这还不是，它们的排列顺序也有要求：首先处理输入；接着实现核心功能；然后处理输出；如有需要，最后处理异常。

无论用哪种方式，`switch`之类的语句都杂糅了太多方法逻辑，使得程序更难调试，使得对象类型绑架了程序逻辑。身为自信优雅的程序员，我们只需要让鸭子叫，然后可以放心的走开。这一位这首先的找出所需的消息和校色，然后确保只有那些真正会叫的“鸭子”才嫩刚进入我们的方法逻辑。

到目前为止，我们见过了直接把值本省作为输入的方式，我们把这种输入称为“简单输入”或“直接输入”。但`Time.now`却是一个间接输入。首先我们引用了`Time`类，然后把`#now`消息发送给`Time`。我们想要的其实是方法`now`的返回值，而不是`Time`常量本身。任何我时候为了得到方法返回值，而向其他对象发送消息，其实就在使用间接输入。

In [2]:
require 'yaml'

def format_time
  user = ENV['USER']
  prefs = YAML.load_file('/home/#{user}/time-prefs.yml')
  format = prefs.fetch('format') {'%D %r'}
  Time.now.strftime(format)
end

:format_time

在此示例中，两个“间接输入”协同工作去得到最终值，而其中的一个间接输入的值之后为了获取另外一个“间接输入”的值。这通常都是软件中`bug`的源头之一。

In [8]:
winners = ["Homestar", "King of Town", "Marzipan", "Strongbad"]

Place = Struct.new(:index, :name, :prize) do
  def to_int
    index
  end
end

first = Place.new(0, "first", "Peasant's Quest game")
second = Place.new(1, "second", "Limozeen Album")
third = Place.new(2, "third", "Butter-da")

[first, second, third].each do |place|
  puts "In #{place.name} place, #{winners[place]}!"
  puts "You win: #{place.prize}"
end

(pry):77: warning: already initialized constant Place
(pry):61: warning: previous definition of Place was here


In first place, Homestar!
You win: Peasant's Quest game
In second place, King of Town!
You win: Limozeen Album
In third place, Marzipan!
You win: Butter-da


[#<struct Place index=0, name="first", prize="Peasant's Quest game">, #<struct Place index=1, name="second", prize="Limozeen Album">, #<struct Place index=2, name="third", prize="Butter-da">]

之所以能这样用，是因为`Ruby`会自动地在数组下标参数上调用`#to_int`方法，使其转换为integer。

In [9]:
class VimConfigFile
  def initialize
    @filename = "#{ENV['HOME']}/.vimrc"
  end
  
  def to_path
    @filename
  end
end

:to_path

In [11]:
vim_config = VimConfigFile.new

File.open(vim_config).lines.count

(pry):105: warning: IO#lines is deprecated; use #each_line instead


689

这是因为，`EmacsConfig`类定义了转换方法`#to_path`，而`File#open`优惠在参数对象上调用`#to_path`方法，以便得到文件名字符串。这样依赖，这个非字符串对象也能工作的很好

In [13]:
nil.to_str

NoMethodError: undefined method `to_str' for nil:NilClass

In [15]:
nil.to_s

""

显示和隐式方式同时存在时，该使用哪一个呢？如果只关心得到预期输入类型而不深究其类型，则可使用显示类型转换方法。

In [19]:
PHONE_EXTENSIONS = ["Operator", "Sales", "Customer Service"]
def dial_extension(dialed_number)
dialed_number = dialed_number.to_i
extension = PHONE_EXTENSIONS[dialed_number]
p "Please hold wihile you are connected to #{extension}"
end


(pry):120: warning: already initialized constant PHONE_EXTENSIONS
(pry):114: warning: previous definition of PHONE_EXTENSIONS was here


:dial_extension

In [20]:
nil.to_i

0

In [22]:
dial_extension(nil)

"Please hold wihile you are connected to Operator"


"Please hold wihile you are connected to Operator"

In [24]:
def set_centrifuge_speed(new_rpm)
  new_rpm = new_rpm.to_int
  p "Adjusting centrifuge to #{new_rmp} RPM"
end

bad_input= nil
set_centrifuge_speed(bad_input)

NoMethodError: undefined method `to_int' for nil:NilClass

to_s是一个显式类型转换方法。显示转换一般用于这样的情形：源类型和目标类型很大程度上不关联

to_str是隐形类型转换方法。适用于元类型和目标类型很相近的情形。